In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 4, Finished, Available)

In [3]:
%%sql 
create table if not exists project_lakehouse.gold_order_return
(
    Order_ID string,
    Return string,
    Order_Year int,
    Order_Month int,
    Create_TS timestamp,
    Modified_TS timestamp
)
using delta
PARTITIONED by (Order_Year, Order_Month )

StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 5, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql
select coalesce(max("Modified_TS"), '1900-01-01')from project_lakehouse.gold_order_return

StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 6, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [6]:
max_date = spark.sql("select coalesce(max('Modified_TS'), '1900-01-01')from project_lakehouse.gold_order_return").first()[0]
max_date

StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 8, Finished, Available)

'1900-01-01'

In [7]:
# Need to created incremental
df = spark.sql(""" select Order_ID  ,
    Return ,
    Order_Year, 
    Order_Month ,
    Create_TS ,
    Modified_TS 
    from project_lakehouse.bronze_sales where Return='Yes' and Modified_TS>'{}' """.format(max_date))


StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 9, Finished, Available)

In [8]:
df.createOrReplaceTempView("ViewReturn")

StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 10, Finished, Available)

In [9]:
%%sql
select * from ViewReturn

StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 11, Finished, Available)

<Spark SQL result set with 123 rows and 6 fields>

In [10]:
%%sql
-- inserting the view into the gold table created above
insert into project_lakehouse.gold_order_return
select * from ViewReturn

StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 12, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
%%sql
-- checking if the data has being inserted sucessfully
select * from project_lakehouse.gold_order_return limit 10


StatementMeta(, d3550837-592e-4c2c-ba67-df6367c2ce5e, 13, Finished, Available)

<Spark SQL result set with 10 rows and 6 fields>